# Youtube comment fetcher

In [ ]:
!pip install azure-eventhub google-api-python-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 11.3 MB/s eta 0:00:00


In [ ]:
from googleapiclient.discovery import build
from transformers import pipeline
from azure.eventhub import EventHubProducerClient, EventData
import json
import time

In [ ]:
YOUTUBE_API_KEY = '**********'
VIDEO_ID = 'fsQgc9pCyDU'
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

In [ ]:
EVENT_HUB_CONNECTION_STR = '*******'
EVENT_HUB_NAME = 'comments-hf'

In [ ]:
producer = EventHubProducerClient.from_connection_string(
    conn_str=EVENT_HUB_CONNECTION_STR,
    eventhub_name=EVENT_HUB_NAME
)

In [ ]:
sentiment_model = pipeline("sentiment-analysis")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def get_all_youtube_comments(video_id, max_total=1000):
    """
    Fetch up to `max_total` comments with pagination,
    returns list of dicts: [{"text": comment_text}, ...]
    """
    comments = []
    next_page_token = None

    while len(comments) < max_total:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat='plainText'
        ).execute()

        for item in response['items']:
            comment_text = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append({"text": comment_text})
            if len(comments) >= max_total:
                break

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    print(f"✅ Collected {len(comments)} comments")
    return comments


In [ ]:
def huggingface_sentiment(text):
    result = sentiment_model(text[:512])[0]  # HuggingFace model max token length safety
    return result['label']

In [ ]:
def rule_based_sentiment(text):
    text = text.lower()
    positive_words = ["good", "love", "great", "amazing", "awesome"]
    negative_words = ["bad", "hate", "terrible", "worst", "awful"]

    if any(word in text for word in positive_words):
        return "Positive"
    elif any(word in text for word in negative_words):
        return "Negative"
    else:
        return "Neutral"

In [ ]:
def send_to_event_hub(comments, batch_size=50):
    from azure.eventhub import EventHubProducerClient, EventData
    import json
    import time

    producer = EventHubProducerClient.from_connection_string(
        conn_str=EVENT_HUB_CONNECTION_STR,
        eventhub_name=EVENT_HUB_NAME
    )

    with producer:
        for i in range(0, len(comments), batch_size):
            batch_comments = comments[i:i + batch_size]
            batch = producer.create_batch()

            for comment in batch_comments:
                text = comment["text"]
                comment["hf_sentiment"] = huggingface_sentiment(text)
                comment["sql_sentiment"] = rule_based_sentiment(text)

                event_data = EventData(json.dumps(comment))

                try:
                    batch.add(event_data)
                except ValueError:
                    # If a single event is too large, skip or log
                    print(f"⚠️ Skipping large comment at index {i}")
                    continue

            try:
                producer.send_batch(batch)
                print(f"✅ Sent batch {i // batch_size + 1} with {len(batch_comments)} comments")
                time.sleep(0.2)  # Optional: slight delay to avoid timeouts
            except Exception as e:
                print(f"❌ Error sending batch {i // batch_size + 1}: {e}")

    print(f"✅ Sent total {len(comments)} comments to Event Hub")



In [ ]:
if __name__ == "__main__":
    VIDEO_ID = "fsQgc9pCyDU"
    MAX_COMMENTS = 13000

    comments = get_all_youtube_comments(VIDEO_ID, max_total=MAX_COMMENTS)
    send_to_event_hub(comments)

✅ Collected 5355 comments
✅ Sent batch 1 with 50 comments
✅ Sent batch 2 with 50 comments
✅ Sent batch 3 with 50 comments
✅ Sent batch 4 with 50 comments
✅ Sent batch 5 with 50 comments
✅ Sent batch 6 with 50 comments
✅ Sent batch 7 with 50 comments
✅ Sent batch 8 with 50 comments
✅ Sent batch 9 with 50 comments
✅ Sent batch 10 with 50 comments
✅ Sent batch 11 with 50 comments
✅ Sent batch 12 with 50 comments
✅ Sent batch 13 with 50 comments
✅ Sent batch 14 with 50 comments
✅ Sent batch 15 with 50 comments
✅ Sent batch 16 with 50 comments
✅ Sent batch 17 with 50 comments
✅ Sent batch 18 with 50 comments
✅ Sent batch 19 with 50 comments
✅ Sent batch 20 with 50 comments
✅ Sent batch 21 with 50 comments
✅ Sent batch 22 with 50 comments
✅ Sent batch 23 with 50 comments
✅ Sent batch 24 with 50 comments
✅ Sent batch 25 with 50 comments
✅ Sent batch 26 with 50 comments
✅ Sent batch 27 with 50 comments
✅ Sent batch 28 with 50 comments
✅ Sent batch 29 with 50 comments
✅ Sent batch 30 with 50 co

In [ ]:
!pip install azure-storage-blob


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 8.5 MB/s eta 0:00:00


# Retrieving CSV from Blob Storage


In [ ]:
from azure.storage.blob import BlobServiceClient
import json
import pandas as pd
import os

# Your storage account connection string
conn_str = "************"

# The container name where Stream Analytics outputs blobs
container_name = "sentimentoutput"

blob_service_client = BlobServiceClient.from_connection_string(conn_str)
container_client = blob_service_client.get_container_client(container_name)

# List all blobs
blob_list = list(container_client.list_blobs())
print(f"✅ Found {len(blob_list)} blobs")

all_data = []

for blob in blob_list:
    blob_client = container_client.get_blob_client(blob)
    blob_data = blob_client.download_blob().readall()
    blob_text = blob_data.decode('utf-8')

    # Parse each line (JSONL)
    for line in blob_text.strip().split('\n'):
        try:
            all_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"⚠️ Could not parse a line in {blob.name}")

# Convert to DataFrame
df = pd.DataFrame(all_data)
print(f"✅ Parsed {len(df)} records")

# Save to CSV
output_csv = "youtube_sentiment_data.csv"
df.to_csv(output_csv, index=False)
print(f"📁 Saved CSV to {os.path.abspath(output_csv)}")


✅ Found 1 blobs
✅ Parsed 5355 records
📁 Saved CSV to /content/youtube_sentiment_data.csv
